In [ ]:
# coding:utf-8
"""
本篇为user_all(),从数据库中读取所有暂存的中间表，连接所有分表，得到用户基础信息的宽表
"""
import pandas as pd
import numpy as np
from impala.dbapi import connect
from impala.util import as_pandas
import cPickle
import re
import datetime as dt
import pymysql.cursors

# Global variables
mysql_host = '172.21.1.151'
mysql_port = 3306
mysql_user = 'xli'
mysql_password = '123456'
mysql_db = 'kdwtemp2'
mysql_charset = 'utf8mb4'

def user_all():
    # 连接数据库以读取中间表
    conn = pymysql.connect(host=mysql_host,
                                      port=mysql_port,
                                      user=mysql_user,
                                      password=mysql_password,
                                      db=mysql_db,
                                      charset=mysql_charset)
    # 1. 读取user_info结果表
    sql = 'SELECT * FROM ljl_user_info'
    user_info = pd.read_sql(sql,conn)
    
    # 2. 读取get_coupon
    sql = 'SELECT * FROM ljl_get_coupon'
    get_coupon = pd.read_sql(sql,conn)
    df = pd.merge(get_coupon,user_info,how='left',left_on='customerid',right_on='customerid')
    
    conn.close()
    return df

In [ ]:
#     # 3.读取use_coupon
#     sql = 'SELECT * FROM ljl_use_coupon'
#     use_coupon = pd.read_sql(sql,conn)
    
#     # 4. 读取destine
#     sql = 'SELECT * FROM ljl_destine'
#     destine = pd.read_sql(sql,conn)
    
#     # 5. 读取tag_age_from_song
#     sql = 'SELECT * FROM ljl_tag_age_from_song'
#     tag_age_from_song = pd.read_sql(sql,conn)
    
#     # 6. 读取shop
#     sql = 'SELECT * FROM ljl_shop'
#     shop = pd.read_sql(sql,conn)
    

In [ ]:
df = user_all()

In [ ]:
pd.value_counts(df['customerid'])

In [ ]:
pd.value_counts(df['kmid'])

In [ ]:
pd.value_counts(df['openid'])

In [ ]:
df.shape

In [10]:
df.columns

Index([u'customerid', u'coupon_get_cnt', u'coupon_get_sum',
       u'source_company_id', u'source_company_cnt', u'get_coupon_type_cnt',
       u'get_com_roomnum', u'get_com_regioncode', u'get_com_managetype',
       u'get_com_vodsystemtype', u'get_coupon_crosscom', u'registertime',
       u'customer_status', u'openid', u'wechat_sex', u'wechat_city',
       u'wechat_province', u'wechat_country', u'kmid', u'member_birthday',
       u'km_sex', u'accountcash', u'accountbalance', u'integralbalance',
       u'consumetotal', u'consumetimes', u'member_age_cut',
       u'lastconsume_diff_create_day', u'lastconsume_diff_create_hour',
       u'lastconsume_diff_now'],
      dtype='object')

In [11]:
df.head()

,customerid,coupon_get_cnt,coupon_get_sum,source_company_id,source_company_cnt,get_coupon_type_cnt,get_com_roomnum,get_com_regioncode,get_com_managetype,get_com_vodsystemtype,...,km_sex,accountcash,accountbalance,integralbalance,consumetotal,consumetimes,member_age_cut,lastconsume_diff_create_day,lastconsume_diff_create_hour,lastconsume_diff_now
0,1003913,12,215,4617,9,4,63,500000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1536143,1,5,4617,1,1,63,500000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3303677,2,20,4617,2,2,63,500000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3931247,4,35,4617,4,4,63,500000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3937144,1,20,4617,1,1,63,500000,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146041 entries, 0 to 146040
Data columns (total 30 columns):
customerid                      145220 non-null object
coupon_get_cnt                  145220 non-null float64
coupon_get_sum                  145220 non-null float64
source_company_id               146041 non-null object
source_company_cnt              145220 non-null float64
get_coupon_type_cnt             145220 non-null float64
get_com_roomnum                 102268 non-null object
get_com_regioncode              102268 non-null object
get_com_managetype              102268 non-null object
get_com_vodsystemtype           102268 non-null object
get_coupon_crosscom             146041 non-null int64
registertime                    129769 non-null object
customer_status                 129769 non-null object
openid                          74296 non-null object
wechat_sex                      65695 non-null object
wechat_city                     65695 non-null object
wechat_pr

In [13]:
df.drop_duplicates('customerid',inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145084 entries, 0 to 145220
Data columns (total 30 columns):
customerid                      145083 non-null object
coupon_get_cnt                  145083 non-null float64
coupon_get_sum                  145083 non-null float64
source_company_id               145084 non-null object
source_company_cnt              145083 non-null float64
get_coupon_type_cnt             145083 non-null float64
get_com_roomnum                 101334 non-null object
get_com_regioncode              101334 non-null object
get_com_managetype              101334 non-null object
get_com_vodsystemtype           101334 non-null object
get_coupon_crosscom             145084 non-null int64
registertime                    129769 non-null object
customer_status                 129769 non-null object
openid                          74159 non-null object
wechat_sex                      65695 non-null object
wechat_city                     65695 non-null object
wechat_pr